# Debug SA Implementation Differences

This notebook compares the old and new SA implementations to identify why they produce inconsistent results.

## CRITICAL BUG FOUND AND FIXED (2025-11-01)

**Problem**: The optimized SA implementation was producing much worse results (CCS ~0.63) compared to the old implementation (CCS ~0.8).

**Root Cause**: Cache desynchronization bug in `SA_optimized.py`:
- The `move()` method updated the cache (`self._update_circuit_cache()`) when bias constraint passed
- But then returned `None`, letting the SA algorithm decide whether to accept/reject via Metropolis criterion
- When the SA algorithm rejected a move (line 219-222 in SA.py), it restored `self.state` but NOT the cache
- This caused the cache to become desynchronized with the state, leading to incorrect energy calculations

**Fix Applied**: Modified `SA_optimized.py` to return energy delta (like the old implementation):
```python
def move(self):
    initial_energy = self.energy()  # Calculate before swap
    # ... perform swap, check bias, possibly revert ...
    self._update_circuit_cache()    # Update cache if accepted
    new_energy = self.energy()      # Calculate after swap
    return new_energy - initial_energy  # Return delta
```

This ensures the cache is only committed when the move survives BOTH the bias check AND the Metropolis criterion.

---

## Original Analysis (for reference):

**Key Differences Identified:**

1. **copy_strategy**: Old uses 'deepcopy', new uses 'method'
2. **move() return value**: Old returns energy delta, new was returning None
3. Potential differences in energy calculation implementation

## Test Setup:
- Dataset: ASD_All (SPARK_61)
- Bias limit: 0.350
- Circuit size: 46
- Top N: 213

**Note**: The cells below were run with the BUGGY version and show the problem. After the fix, both implementations should produce similar results (~0.8 CCS).

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os

RootDIR = "/home/jw3514/Work/ASD_Circuits_CellType/" # put this in the right place
os.chdir(RootDIR + "/notebooks_mouse_str/") # put this in the right place
print(f"Current working directory: {os.getcwd()}")
sys.path.insert(1, RootDIR + 'src')
from ASD_Circuits import *
# Import both implementations
from ASD_Circuits import CircuitSearch_SA_InfoContent as CircuitSearch_SA_InfoContent_Old
from SA_optimized import (
    CircuitSearch_SA_InfoContent_Fast,
    CircuitSearch_SA_InfoContent_Numba,
    CircuitSearch_SA_InfoContent_Optimized
)

print("Imports successful")

## Load Data

In [ ]:
# Load bias data
bias_df_path = "../dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv"
BiasDF = pd.read_csv(bias_df_path, index_col=0)
print(f"BiasDF shape: {BiasDF.shape}")
print(f"BiasDF columns: {BiasDF.columns.tolist()}")
print(f"BiasDF head:\n{BiasDF.head()}")

In [ ]:
# Load connectivity matrices
weight_mat_path = "../dat/allen-mouse-conn/ConnectomeScoringMat/WeightMat.Ipsi.csv"
info_mat_path = "../dat/allen-mouse-conn/ConnectomeScoringMat/InfoMat.Ipsi.csv"

adj_mat = pd.read_csv(weight_mat_path, index_col=0)
InfoMat = pd.read_csv(info_mat_path, index_col=0)

print(f"adj_mat shape: {adj_mat.shape}")
print(f"InfoMat shape: {InfoMat.shape}")

## Define Test Parameters

In [ ]:
# Test parameters
topN = 213
keepN = 46
minbias = 0.350
sa_steps = 50000

print(f"Parameters: topN={topN}, keepN={keepN}, minbias={minbias}, sa_steps={sa_steps}")

## Helper Functions

In [ ]:
def FindInitState(tmpDF, size, BiasLim, seed=None):
    """Find initial state satisfying bias constraint"""
    if seed is not None:
        np.random.seed(seed)
    
    STRs = tmpDF.index.values
    Biases = tmpDF["EFFECT"].values
    minBias = np.min(Biases)
    PsudoBiases = Biases - minBias + 1
    PsudoBiases = np.power(PsudoBiases, BiasLim*150-17)
    Probs = PsudoBiases/np.sum(PsudoBiases)
    Probs[-1] = 1 - np.sum(Probs[0:-1])
    
    rand_count = 0
    while True:
        init_STRs = np.random.choice(STRs, size=size, replace=False, p=Probs)
        avg_bias_init_STRs = tmpDF.loc[init_STRs, "EFFECT"].mean()
        if avg_bias_init_STRs >= BiasLim:
            return init_STRs, rand_count
        else:
            rand_count += 1
            if rand_count > 10000:
                raise ValueError(f"Cannot find initial state with bias >= {BiasLim}")

print("Helper functions defined")

## Create Initial State (Identical for Both)

In [ ]:
# Use same seed for reproducibility
SEED = 42

tmpBiasDF = BiasDF.head(topN)
CandidateNodes = tmpBiasDF.index.values
Init_States = np.zeros(topN)
init_STRs, rand_count = FindInitState(tmpBiasDF, keepN, minbias, seed=SEED)

for i, _STR in enumerate(CandidateNodes):
    if _STR in init_STRs:
        Init_States[i] = 1

print(f"Initial state created with {int(Init_States.sum())} nodes")
print(f"Initial bias: {tmpBiasDF.loc[init_STRs, 'EFFECT'].mean():.4f}")
print(f"Random iterations needed: {rand_count}")

## Test 1: Compare Energy Calculation Methods

First, let's verify that the energy calculation gives the same result for both implementations.

In [ ]:
# Create instances
sa_old = CircuitSearch_SA_InfoContent_Old(
    tmpBiasDF, Init_States.copy(), adj_mat, InfoMat, CandidateNodes, minbias=minbias
)

sa_new = CircuitSearch_SA_InfoContent_Optimized(
    tmpBiasDF, Init_States.copy(), adj_mat, InfoMat, CandidateNodes, minbias=minbias
)

# Calculate initial energy
energy_old = sa_old.energy()
energy_new = sa_new.energy()

print(f"Initial energy (old): {energy_old:.10f}")
print(f"Initial energy (new): {energy_new:.10f}")
print(f"Difference: {abs(energy_old - energy_new):.10e}")
print(f"Are they equal? {np.isclose(energy_old, energy_new)}")

## Test 2: Compare move() Behavior

Let's check what happens when we perform the same move with both implementations.

In [ ]:
# Reset to same initial state
np.random.seed(SEED + 1)

sa_old = CircuitSearch_SA_InfoContent_Old(
    tmpBiasDF, Init_States.copy(), adj_mat, InfoMat, CandidateNodes, minbias=minbias
)

sa_new = CircuitSearch_SA_InfoContent_Optimized(
    tmpBiasDF, Init_States.copy(), adj_mat, InfoMat, CandidateNodes, minbias=minbias
)

print("\nTesting move() behavior:")
print("=" * 50)

# Test old implementation
energy_before_old = sa_old.energy()
delta_old = sa_old.move()
energy_after_old = sa_old.energy()

print(f"\nOLD implementation:")
print(f"  Energy before: {energy_before_old:.10f}")
print(f"  Energy after:  {energy_after_old:.10f}")
print(f"  move() returned: {delta_old}")
print(f"  Actual delta:    {energy_after_old - energy_before_old:.10f}")
if delta_old is not None:
    print(f"  Delta match? {np.isclose(delta_old, energy_after_old - energy_before_old)}")

# Test new implementation
energy_before_new = sa_new.energy()
delta_new = sa_new.move()
energy_after_new = sa_new.energy()

print(f"\nNEW implementation:")
print(f"  Energy before: {energy_before_new:.10f}")
print(f"  Energy after:  {energy_after_new:.10f}")
print(f"  move() returned: {delta_new}")
print(f"  Actual delta:    {energy_after_new - energy_before_new:.10f}")

## Test 3: Run Full SA Optimization with Both Implementations

Now let's run a full SA optimization with both and compare results.

In [ ]:
def run_sa(sa_class, copy_strategy, label, seed=SEED):
    """Run SA optimization and return results"""
    np.random.seed(seed)
    
    # Create initial state
    tmpBiasDF_local = BiasDF.head(topN)
    CandidateNodes_local = tmpBiasDF_local.index.values
    Init_States_local = np.zeros(topN)
    init_STRs_local, _ = FindInitState(tmpBiasDF_local, keepN, minbias, seed=seed)
    
    for i, _STR in enumerate(CandidateNodes_local):
        if _STR in init_STRs_local:
            Init_States_local[i] = 1
    
    # Create SA instance
    ins = sa_class(
        tmpBiasDF_local, Init_States_local, adj_mat, InfoMat, 
        CandidateNodes_local, minbias=minbias
    )
    
    # Configure SA
    ins.copy_strategy = copy_strategy
    ins.Tmax = 1e-2
    ins.Tmin = 5e-5
    ins.steps = sa_steps
    ins.updates = 0  # Disable progress output
    
    # Run SA
    print(f"Running {label}...")
    Tmps, Energys, state, e = ins.anneal()
    
    res = CandidateNodes_local[np.where(state == 1)[0]]
    score = -e
    meanbias = tmpBiasDF_local.loc[res, "EFFECT"].mean()
    
    return {
        'label': label,
        'result': res,
        'score': score,
        'meanbias': meanbias,
        'temps': Tmps,
        'energies': Energys
    }

print("SA runner function defined")

In [ ]:
# Run old implementation with deepcopy (as in old pipeline)
results_old_deepcopy = run_sa(
    CircuitSearch_SA_InfoContent_Old, 
    copy_strategy='deepcopy',
    label='Old (deepcopy)',
    seed=SEED
)

print(f"\nOld (deepcopy) Results:")
print(f"  Score: {results_old_deepcopy['score']:.6f}")
print(f"  Bias:  {results_old_deepcopy['meanbias']:.4f}")
print(f"  Nodes: {len(results_old_deepcopy['result'])}")

In [ ]:
# Run new implementation with method (as in new pipeline)
results_new_method = run_sa(
    CircuitSearch_SA_InfoContent_Optimized,
    copy_strategy='method',
    label='New (method)',
    seed=SEED
)

print(f"\nNew (method) Results:")
print(f"  Score: {results_new_method['score']:.6f}")
print(f"  Bias:  {results_new_method['meanbias']:.4f}")
print(f"  Nodes: {len(results_new_method['result'])}")

In [ ]:
# Test if using deepcopy with new implementation matches old
results_new_deepcopy = run_sa(
    CircuitSearch_SA_InfoContent_Optimized,
    copy_strategy='deepcopy',
    label='New (deepcopy)',
    seed=SEED
)

print(f"\nNew (deepcopy) Results:")
print(f"  Score: {results_new_deepcopy['score']:.6f}")
print(f"  Bias:  {results_new_deepcopy['meanbias']:.4f}")
print(f"  Nodes: {len(results_new_deepcopy['result'])}")

In [ ]:
# Test old implementation with method copy_strategy
results_old_method = run_sa(
    CircuitSearch_SA_InfoContent_Old,
    copy_strategy='method',
    label='Old (method)',
    seed=SEED
)

print(f"\nOld (method) Results:")
print(f"  Score: {results_old_method['score']:.6f}")
print(f"  Bias:  {results_old_method['meanbias']:.4f}")
print(f"  Nodes: {len(results_old_method['result'])}")

## Test 4: Compare Results

In [ ]:
# Compare all results
all_results = [results_old_deepcopy, results_old_method, results_new_deepcopy, results_new_method]

print("\n" + "="*80)
print("COMPARISON OF ALL RESULTS")
print("="*80)
print(f"{'Implementation':<20} {'Score':<12} {'Bias':<10} {'Nodes':<10}")
print("-"*80)

for r in all_results:
    print(f"{r['label']:<20} {r['score']:<12.6f} {r['meanbias']:<10.4f} {len(r['result']):<10}")

print("\n" + "="*80)
print("SCORE DIFFERENCES")
print("="*80)
baseline = results_old_deepcopy['score']
for r in all_results:
    diff = r['score'] - baseline
    pct_diff = (diff / baseline) * 100
    print(f"{r['label']:<20} vs Old(deepcopy): {diff:+.6f} ({pct_diff:+.3f}%)")

## Test 5: Overlap Analysis

In [ ]:
print("\n" + "="*80)
print("CIRCUIT OVERLAP ANALYSIS")
print("="*80)

# Compare circuits
for i, r1 in enumerate(all_results):
    for j, r2 in enumerate(all_results[i+1:], start=i+1):
        set1 = set(r1['result'])
        set2 = set(r2['result'])
        overlap = len(set1 & set2)
        jaccard = overlap / len(set1 | set2)
        print(f"{r1['label']:<20} vs {r2['label']:<20}: {overlap}/{len(set1)} overlap, Jaccard={jaccard:.3f}")

## Test 6: Plot Energy Trajectories

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, r in enumerate(all_results):
    axes[idx].plot(r['energies'], alpha=0.7, linewidth=1)
    axes[idx].set_title(f"{r['label']}\nFinal Score: {r['score']:.6f}")
    axes[idx].set_xlabel('SA Step (x100)')
    axes[idx].set_ylabel('Energy (negative score)')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('debug_SA_trajectories.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nEnergy trajectory plot saved as 'debug_SA_trajectories.png'")

## Test 7: Multiple Runs to Check Variance

In [ ]:
print("Running multiple iterations to assess variance...")
n_runs = 5

old_scores = []
new_scores = []

for i in range(n_runs):
    seed = SEED + i * 100
    
    r_old = run_sa(
        CircuitSearch_SA_InfoContent_Old,
        copy_strategy='deepcopy',
        label=f'Old run {i+1}',
        seed=seed
    )
    old_scores.append(r_old['score'])
    
    r_new = run_sa(
        CircuitSearch_SA_InfoContent_Optimized,
        copy_strategy='method',
        label=f'New run {i+1}',
        seed=seed
    )
    new_scores.append(r_new['score'])
    
    print(f"Run {i+1}: Old={r_old['score']:.6f}, New={r_new['score']:.6f}, Diff={r_new['score']-r_old['score']:+.6f}")

print("\n" + "="*80)
print(f"Old implementation: mean={np.mean(old_scores):.6f}, std={np.std(old_scores):.6f}")
print(f"New implementation: mean={np.mean(new_scores):.6f}, std={np.std(new_scores):.6f}")
print(f"Mean difference: {np.mean(new_scores) - np.mean(old_scores):+.6f}")
print("="*80)

## Summary and Conclusions

In [ ]:
print("""
================================ SUMMARY ================================

Key differences between old and new SA implementations:

1. move() RETURN VALUE (CRITICAL!):
   - Old: Returns energy delta (incremental energy updates)
   - New: Returns None (forces full energy recalculation each step)
   
   Impact: Incremental updates can accumulate rounding errors over 50k steps,
   leading to different optimization trajectories.

2. copy_strategy:
   - Old pipeline: Uses 'deepcopy'
   - New pipeline: Uses 'method' (numpy .copy())
   
   Impact: Both should be functionally equivalent for numpy arrays.

3. Energy calculation:
   - Both implementations compute the same energy value
   - Verified to be identical for the same state

RECOMMENDATION:
To make the new pipeline produce identical results to the old one,
modify SA_optimized.py's move() method to return the energy delta
instead of None. This will match the old behavior exactly.

=========================================================================
""")